In [2]:
import os
import torch
import hydra
import lightning.pytorch as pl
from lightning.pytorch.callbacks import LearningRateMonitor
from puzzlefusion_plusplus.denoiser.dataset.dataset import build_geometry_dataloader
import importlib
from puzzlefusion_plusplus.denoiser.dataset import dataset
from omegaconf import OmegaConf,open_dict
import omegaconf

config_home_dir = './config'


data_home_dir = '/disk2/data/breaking_bad/'
data_home_dir = '/disk2/data/shape_dataset/'
data_type_name = 'shape'



cfg_denoiser_data = omegaconf.OmegaConf.load(config_home_dir+'/denoiser/data.yaml')
cfg_denoiser_encode = omegaconf.OmegaConf.load(config_home_dir+'/denoiser/encoder.yaml')
cfg_denoiser_global_config = omegaconf.OmegaConf.load(config_home_dir+'/denoiser/global_config.yaml')
cfg_denoiser_model = omegaconf.OmegaConf.load(config_home_dir+'/denoiser/model.yaml')

cfg = OmegaConf.merge(

    cfg_denoiser_data,
    cfg_denoiser_encode,
    cfg_denoiser_global_config,
    cfg_denoiser_model
)



cfg.data.data_dir = data_home_dir+f'data/pc_data/{data_type_name}/train/'
cfg.data.data_val_dir = data_home_dir+f'data/pc_data/{data_type_name}/val/'
cfg.data.mesh_data_dir = data_home_dir+'data/'
cfg.data.data_fn = data_type_name+".{}.txt"
cfg.data.batch_size = 3
cfg.data.val_batch_size= 3

cfg.experiment_name = 'shape_epoch10'
cfg.model.encoder_weights_path =  f'{data_home_dir}/output/autoencoder/{cfg.experiment_name}'+'/training/last.ckpt'


cfg.ckpt_path= None
cfg.experiment_output_path = data_home_dir+'output/denoiser/${experiment_name}/'
cfg.trainer.max_epochs =  3
cfg.trainer.check_val_every_n_epoch =  1


cfg.trainer.strategy='ddp'
# In[4]:


def init_callbacks(cfg):
    checkpoint_monitor = hydra.utils.instantiate(cfg.checkpoint_monitor)
    lr_monitor = LearningRateMonitor(logging_interval="epoch")
    # print_callback = PrintCallback()
    return [checkpoint_monitor, lr_monitor]



In [3]:
# fix the seed
pl.seed_everything(cfg.train_seed, workers=True)

# create directories for training outputs
os.makedirs(os.path.join(cfg.experiment_output_path, "training"), exist_ok=True)



Seed set to 123


tensor([[ 1.0827,  0.0000,  0.0000, -1.2239],
        [ 0.7017,  0.0000,  0.0000,  1.0541]])

In [20]:
import importlib
import puzzlefusion_plusplus.denoiser.dataset.dataset
importlib.reload(puzzlefusion_plusplus.denoiser.dataset.dataset)
from puzzlefusion_plusplus.denoiser.dataset.dataset import build_geometry_dataloader


# initialize data
train_loader, val_loader = build_geometry_dataloader(cfg)

100%|██████████| 34/34 [00:00<00:00, 428.74it/s]


In [21]:
_item = None
train_count = 0
val_count = 0
for nth_batch, batch in enumerate(train_loader):
    print(nth_batch, len(batch), batch.keys())
    for _i in range(len(batch['mesh_file_path'])):
        
        print(batch["data_id"][_i], batch["mesh_file_path"][_i])
        #_item_gt = batch["part_pcs_gt"][_i].tolist()
        #_item_rt = batch["part_pcs"][_i].tolist()
        print('part_trans',batch["part_trans"][_i].shape,batch["part_trans"][_i])
        print('noise_rots',batch["noise_rots"][_i].shape,batch["noise_rots"][_i])
        #print(_item)
        #np.savez(f'{data_home_dir}/data/test_data/{batch["data_id"][_i]}_gt.npz', _item_gt) 
        #np.savez(f'{data_home_dir}/data/test_data/{batch["data_id"][_i]}_rt.npz', _item_rt) 
        if True:
            break
    if True:
        break


part_trans_ref_(1, 3)_[[0.04969739 0.         0.36612418]]


part_rots_ref_(1, 4)_[[0.95328075 0.         0.05462699 0.2971056 ]]


part_trans_ref_(0, 3)_[]

part_trans_ref_(0, 3)_[]

part_trans_ref_(1, 3)_[[0.6318365  0.         0.19201863]]

noise_trans_torch.Size([1, 3])_[[ 0.3235783   0.         -0.43470618]]

part_trans_ref_(1, 3)_[[0.44563684 0.         0.74065906]]



part_trans_ref_(0, 3)_[]

part_trans_ref_(1, 3)_[[0.74015707 0.         0.895541  ]]



part_rots_ref_(0, 4)_[]


part_rots_ref_(0, 4)_[]




part_rots_ref_(1, 4)_[[0.6140593 0.7892599 0.        0.       ]]

noise_rots_torch.Size([1, 4])_[[-1.3708097   0.          0.          0.23587698]]

part_rots_ref_(1, 4)_[[0.6168895 0.        0.7870498 0.       ]]


part_trans_ref_(0, 3)_[]

part_rots_ref_(1, 4)_[[0.17037436 0.         0.9637981  0.20509948]]

part_rots_ref_(0, 4)_[]





noise_trans_torch.Size([0, 3])_[]


part_trans_ref_(0, 3)_[]


noise_trans_torch.Size([0, 3])_[]


noise_trans_torch.Size([1, 3])_[[-0.454

KeyError: 'noise_rots'



noise_trans_torch.Size([0, 3])_[]


noise_rots_torch.Size([0, 4])_[]


part_trans_ref_(0, 3)_[]


part_rots_ref_(0, 4)_[]


noise_trans_torch.Size([0, 3])_[]


noise_rots_torch.Size([0, 4])_[]



In [ ]:

# initialize model
model = hydra.utils.instantiate(cfg.model.model_name, cfg)

if cfg.model.encoder_weights_path is not None:
    encoder_weights = torch.load(cfg.model.encoder_weights_path)['state_dict']
    model.encoder.load_state_dict({k.replace('ae.', ''): v for k, v in encoder_weights.items()})
    # freeze the encoder
    for param in model.encoder.parameters():
        param.requires_grad = False

# initialize logger
logger = hydra.utils.instantiate(cfg.logger)

# initialize callbacks
callbacks = init_callbacks(cfg)

# initialize trainer
trainer = pl.Trainer(
    callbacks=callbacks,
    logger=logger,
    **cfg.trainer
)

# check the checkpoint
if cfg.ckpt_path is not None:
    assert os.path.exists(cfg.ckpt_path), "Error: Checkpoint path does not exist."

# start training
trainer.fit(
    model=model, 
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    ckpt_path=cfg.ckpt_path
)



#77222a952435b1b516e70facc0fd8554f280f918
main(cfg)


100%|██████████| 34/34 [00:00<00:00, 445.07it/s]


[0.0023310933697341246, 0.0015830856048894128, 0.9999960299139538][-0.004443423627300274, 0.0013972501207013242, 0.9999891517804423]

[-0.0004559545197791059, -0.0029019871773632686, 0.9999956852786409][-0.24204741158538035, 0.9700860169589446, -0.018605650903624166][-0.3679854355984607, 0.9271458227198749, -0.07062112007393574][-0.0011881504254301878, 0.003811472701271705, 0.9999920304554502][0.38377062836926307, -0.019864694854441884, 0.9232147630423817][0.0012962371559663572, -0.00015562117196977362, 0.99999914777528]


[0.000529146338763008, -0.0015071868560194946, 0.9999987241951529]

[-0.020661201355968672, 0.3163161190240201, 0.9484288205258791]


[0.0017172670171577413, 0.00039634059636881275, 0.9999984469528558][5.2182471482640205e-05, 0.0001546709254816406, 0.9999999866769472][-0.0017905376519198597, 0.001005341442517636, 0.9999978916295278]


[-0.00046062570703669974, -0.00437929463715628, 0.9999903047542207][-0.0007356788925982891, 0.000404774382868923, 0.9999996474670708][

In [4]:
cfg.model.encoder_weights_path = cfg.experiment_output_path+'/training/last.ckpt'
cfg.ckpt_path= None
cfg.experiment_output_path = data_home_dir+'output/denoiser/${experiment_name}/'

cfg.trainer.strategy='ddp'

print(OmegaConf.to_yaml(cfg))



data:
  batch_size: 3
  val_batch_size: 3
  num_workers: 10
  data_fn: shape.{}.txt
  data_dir: /disk2/data/shape_dataset/data/pc_data/shape/train/
  data_val_dir: /disk2/data/shape_dataset/data/pc_data/shape/val/
  mesh_data_dir: /disk2/data/shape_dataset/data/
  rot_range: -1
  overfit: -1
  data_keys:
  - part_ids
  num_pc_points: 1000
  min_num_part: 2
  max_num_part: 20
  shuffle_parts: false
  category: all
ae:
  ae_name:
    _target_: puzzlefusion_plusplus.denoiser.model.modules.encoder.VQVAE
  n_embeddings: 1024
  embedding_dim: 16
  num_point: 25
  num_dim: 64
  local_decode_pts: 40
  beta: 0.25
hydra:
  output_subdir: null
  run:
    dir: .
defaults:
- _self_
- encoder
- data
- model
- override hydra/hydra_logging: disabled
- override hydra/job_logging: disabled
project_root_path: /disk2/data/shape_dataset/
experiment_output_path: /disk2/data/shape_dataset/output/denoiser/${experiment_name}/
ckpt_path: null
experiment_name: shape_epoch10
train_seed: 123
test_seed: 123
logger:

In [5]:
from puzzlefusion_plusplus.denoiser.dataset.dataset import build_geometry_dataloader as denoiser_dataloader

In [6]:
pl.seed_everything(cfg.train_seed, workers=True)

# create directories for training outputs
os.makedirs(os.path.join(cfg.experiment_output_path, "training"), exist_ok=True)

# initialize data
train_loader, val_loader = denoiser_dataloader(cfg)



Seed set to 123
100%|██████████| 34/34 [00:00<00:00, 562.25it/s]


In [5]:
for nth_batch, batch in enumerate(train_loader):
    print(nth_batch, batch['mesh_file_path'], batch['data_id'])

NameError: name 'train_loader' is not defined

In [ ]:

# initialize model
model = hydra.utils.instantiate(cfg.model.model_name, cfg)

if cfg.model.encoder_weights_path is not None:
    encoder_weights = torch.load(cfg.model.encoder_weights_path)['state_dict']
    model.encoder.load_state_dict({k.replace('ae.', ''): v for k, v in encoder_weights.items()})
    # freeze the encoder
    for param in model.encoder.parameters():
        param.requires_grad = False

# initialize logger
logger = hydra.utils.instantiate(cfg.logger)

# initialize callbacks
callbacks = init_callbacks(cfg)

# initialize trainer
trainer = pl.Trainer(
    callbacks=callbacks,
    logger=logger,
    **cfg.trainer
)

# check the checkpoint
if cfg.ckpt_path is not None:
    assert os.path.exists(cfg.ckpt_path), "Error: Checkpoint path does not exist."

# start training
trainer.fit(
    model=model, 
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    ckpt_path=cfg.ckpt_path
)